# Charger les bibs

In [23]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras import optimizers
from PIL import Image
import numpy as np
import glob
import os


# Paramétres de l'inférence

In [24]:
model_path = './Modelcpp1.h5'
img_path = 'C:/Users/adm_ael-filou/Desktop/Exemple_Anis/prediction/2.png'
folder_path ='./all/'
folder_new_img_path = "./prediction/"
pred =['Compound Switch','Oblique Crossing','Single Switch']


# Charger et compiler le modéle

In [25]:
def load_and_compile(model_path):
    
    # load the trained model
    opt = optimizers.Adam(learning_rate =0.001)
    model = load_model(model_path)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# Pré traitement des images

In [26]:
def pre_teret2(img,min_size=256, fill_color=(0, 0, 0)):
    
    old_size = img.size  # old_size[0] is in (width, height) format
    ratio = float(min_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # thumbnail is a in-place operation
    img = img.resize(new_size)
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (min_size, min_size))
    new_im.paste(img, ((min_size-new_size[0])//2,
                        (min_size-new_size[1])//2))
        
    return new_im

In [27]:
model = load_and_compile(model_path)

# Inférer une seule image

In [28]:
def infer_one_img(model,pred,img_path):
    test_image = pre_teret2(Image.open(img_path))
    #test_image.show()
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    return(pred[np.argmax(result)])
res = infer_one_img(model,pred,img_path)
print(res)

Single Switch


# Inférer les dossiers test

In [15]:
def infer_test_folders(model,pred,folder_path):
    for i in pred:
        print(f"-------les images du dossier {i} --------------")
        for file in glob.iglob(os.path.join(folder_path+i,"*.png")):
            test_image = pre_teret2(Image.open(file))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = model.predict(test_image)
            print(f"here the result {result} and the pred {pred[np.argmax(result)]}")
            if (pred[np.argmax(result)] != i):
                print(file)
                print(f"fausse préduction p = {pred[np.argmax(result)]} GT = {i}")
                print(result)
                print("----------------------------------------------------------")
infer_test_folders(model,pred,folder_path)

-------les images du dossier Compound Switch --------------
-------les images du dossier Oblique Crossing --------------
-------les images du dossier Single Switch --------------


In [16]:
['Compound Switch','Oblique Crossing','Single Switch']

['Compound Switch', 'Oblique Crossing', 'Single Switch']

# Inférer les nouvelles images

In [6]:
def infer_new_imgs(model,pred,folder_new_img_path):
    for file in glob.iglob(os.path.join(folder_new_img_path,"*.png")):
        test_image = pre_teret2(Image.open(file))
        #test_image.show()
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        print("----------------------------------------------------------")
        print(file)
        print(f"here the result {result} and the pred {pred[np.argmax(result)]}")
infer_new_imgs(model,pred,folder_new_img_path)

----------------------------------------------------------
./prediction\1.png
here the result [[9.9734324e-01 5.3560365e-09 2.6568095e-03]] and the pred Compound Switch
----------------------------------------------------------
./prediction\10.png
here the result [[2.0820050e-07 5.1755596e-05 9.9994802e-01]] and the pred Single Switch
----------------------------------------------------------
./prediction\11.png
here the result [[3.8961221e-07 9.9485767e-01 5.1419646e-03]] and the pred Oblique Crossing
----------------------------------------------------------
./prediction\12.png
here the result [[1.9345637e-07 7.4223476e-06 9.9999237e-01]] and the pred Single Switch
----------------------------------------------------------
./prediction\13.png
here the result [[3.1027639e-06 4.4507154e-09 9.9999690e-01]] and the pred Single Switch
----------------------------------------------------------
./prediction\14.png
here the result [[1.06591125e-10 1.00000000e+00 1.35810598e-08]] and the pred

In [1]:
from PIL import Image, ImageOps
import cv2
import numpy as np 

def pretraitement (img,desired_size = 256):
     old_size = img.shape[:2] # old_size is in (height, width) format
     ratio = float(desired_size)/max(old_size)
     new_size = tuple([int(x*ratio) for x in old_size])
     img = cv2.resize(img, (new_size[1], new_size[0]))
     delta_w = desired_size - new_size[1]
     delta_h = desired_size - new_size[0]
     top, bottom = delta_h//2, delta_h-(delta_h//2)
     left, right = delta_w//2, delta_w-(delta_w//2)
     color = [0, 0, 0]
     new_im = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
     return new_im


In [2]:
image = pretraitement(cv2.imread('C:/Users/adm_ael-filou/Desktop/Exemple_Anis/prediction/22.png'))

In [3]:
image.shape


(256, 256, 3)

In [61]:
import tensorflow as tf
import numpy as np
def recognize(jpg_path, pb_file_path):
  with tf.Graph().as_default():
    output_graph_def = tf.compat.v1.GraphDef()
    with open(pb_file_path, 'rb') as f:
      output_graph_def.ParseFromString(f.read())
      tensors = tf.import_graph_def(output_graph_def, name='')
      print('tensors: ', tensors)
    with tf.compat.v1.Session() as sess:
      init = tf.compat.v1.global_variables_initializer()
      sess.run(init)
      op = sess.graph.get_operations()
      for i, m in enumerate(op):
        print('op{}: '.format(i), m.values())
      input_x = sess.graph.get_tensor_by_name('yourInputName:0')
      print('input_X: ', input_x)
      out_softmax = sess.graph.get_tensor_by_name('sequential/dense_1/Softmax:0')
      print('Output:', out_softmax)
      img = cv2.imread(jpg_path)
      img = cv2.resize(img, (256, 256))
      img = img.astype(np.float32)
      #img = 1-img/255
      print('img data type: ', img.dtype)
      for row in range(256):
        for col in range(256):
          if col!=255:
            print(img[row][col], ' ', end='')
          else:
            print(img[row][col])
      img_out_softmax = sess.run(out_softmax, feed_dict={input_x: np.reshape(img, (1, 256, 256, 3))})
      print('img_out_softmax: ', img_out_softmax)
      for i, prob in enumerate(img_out_softmax[0]):
        print('class{} prob: {}'.format(i, prob))
      prediction_labels = np.argmax(img_out_softmax, axis=1)
      print('Final class is: ', prediction_labels)
      print('prob of label: ', img_out_softmax[0, prediction_labels])
pb_path = './frozen_models/frozen_graph.pb'
img = './1.png'
recognize(img, pb_path)

tensors:  None
op0:  (<tf.Tensor 'yourInputName:0' shape=(None, 256, 256, 3) dtype=float32>,)
op1:  (<tf.Tensor 'sequential/conv2d/Conv2D/ReadVariableOp/resource:0' shape=(3, 3, 3, 16) dtype=float32>,)
op2:  (<tf.Tensor 'sequential/conv2d/Conv2D/ReadVariableOp:0' shape=(3, 3, 3, 16) dtype=float32>,)
op3:  (<tf.Tensor 'sequential/conv2d/Conv2D:0' shape=(None, 254, 254, 16) dtype=float32>,)
op4:  (<tf.Tensor 'sequential/conv2d/BiasAdd/ReadVariableOp/resource:0' shape=(16,) dtype=float32>,)
op5:  (<tf.Tensor 'sequential/conv2d/BiasAdd/ReadVariableOp:0' shape=(16,) dtype=float32>,)
op6:  (<tf.Tensor 'sequential/conv2d/BiasAdd:0' shape=(None, 254, 254, 16) dtype=float32>,)
op7:  (<tf.Tensor 'sequential/conv2d/Relu:0' shape=(None, 254, 254, 16) dtype=float32>,)
op8:  (<tf.Tensor 'sequential/max_pooling2d/MaxPool:0' shape=(None, 127, 127, 16) dtype=float32>,)
op9:  (<tf.Tensor 'sequential/batch_normalization/ReadVariableOp/resource:0' shape=(16,) dtype=float32>,)
op10:  (<tf.Tensor 'sequentia

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

...

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

[0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 0. 0.]  [0. 

In [1]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)
from keras.models import load_model
model = load_model('./ModelFinale.h5')

C:\Users\adm_ael-filou\AppData\Roaming\Python\Python38\site-packages\keras\backend.py:400: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [12]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np


model = keras.models.load_model('./ModelFinale.h5')

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

# inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
# serialize the frozen graph and its text representation to disk.
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="simple_frozen_graph.pb",
                  as_text=False)

#Optional
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="simple_frozen_graph.pbtxt",
                as_text=True)

model.summary()


--------------------------------------------------
Frozen model layers: 
x
sequential/conv2d/Conv2D/ReadVariableOp/resource
sequential/conv2d/Conv2D/ReadVariableOp
sequential/conv2d/Conv2D
sequential/conv2d/BiasAdd/ReadVariableOp/resource
sequential/conv2d/BiasAdd/ReadVariableOp
sequential/conv2d/BiasAdd
sequential/conv2d/Relu
sequential/max_pooling2d/MaxPool
sequential/batch_normalization/ReadVariableOp/resource
sequential/batch_normalization/ReadVariableOp
sequential/batch_normalization/ReadVariableOp_1/resource
sequential/batch_normalization/ReadVariableOp_1
sequential/batch_normalization/FusedBatchNormV3/ReadVariableOp/resource
sequential/batch_normalization/FusedBatchNormV3/ReadVariableOp
sequential/batch_normalization/FusedBatchNormV3/ReadVariableOp_1/resource
sequential/batch_normalization/FusedBatchNormV3/ReadVariableOp_1
sequential/batch_normalization/FusedBatchNormV3
sequential/conv2d_1/Conv2D/ReadVariableOp/resource
sequential/conv2d_1/Conv2D/ReadVariableOp
sequential/conv2d

In [1]:
from keras.models import load_model
model = load_model('./keras_model.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'dense_1')>]
[<KerasTensor: shape=(None, 256, 256, 3) dtype=float32 (created by layer 'conv2d_input')>]


In [2]:
from keras.models import load_model
model = load_model('./ModelFinale.h5')
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [3]:

model.save('test.h5', include_optimizer=False)

